In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [6]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                 'rnd_padding', 
                 'aug_padding'
                "zoom_padding"
]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/1conv/64filts_size5/' #'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 64 #None
kernel_size = [5] #None
pool_size = None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
#architecture = "only_denses"
architecture = "conv_dense"

### Loading stuff

In [3]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [4]:
#model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 313)         20345     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 76)          0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_4 (Dense)              (None, 1000, 313)         20345     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 76)          0         
__________

138/138 [==============================] - 4s 28ms/step - loss: 0.7186 - acc: 0.1865 - val_loss: 0.2785 - val_acc: 0.1003

Epoch 00034: val_acc did not improve from 0.82056
Epoch 35/200
138/138 [==============================] - 4s 28ms/step - loss: 0.6934 - acc: 0.1942 - val_loss: 0.2561 - val_acc: 0.1162

Epoch 00035: val_acc did not improve from 0.82056
Epoch 36/200
138/138 [==============================] - 4s 28ms/step - loss: 0.6709 - acc: 0.2174 - val_loss: 0.2395 - val_acc: 0.1405

Epoch 00036: val_acc did not improve from 0.82056
Epoch 37/200
138/138 [==============================] - 4s 28ms/step - loss: 0.6423 - acc: 0.2264 - val_loss: 0.2262 - val_acc: 0.1622

Epoch 00037: val_acc did not improve from 0.82056
Epoch 38/200
138/138 [==============================] - 4s 28ms/step - loss: 0.6269 - acc: 0.2336 - val_loss: 0.2465 - val_acc: 0.1130

Epoch 00038: val_acc did not improve from 0.82056
Epoch 39/200
138/138 [==============================] - 4s 28ms/step - loss: 0.6020

138/138 [==============================] - 4s 28ms/step - loss: 0.1402 - acc: 0.4318 - val_loss: 0.0293 - val_acc: 0.2299

Epoch 00078: val_acc did not improve from 0.82056
Epoch 79/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1387 - acc: 0.4403 - val_loss: 0.0294 - val_acc: 0.2235

Epoch 00079: val_acc did not improve from 0.82056
Epoch 80/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1333 - acc: 0.4369 - val_loss: 0.0227 - val_acc: 0.2158

Epoch 00080: val_acc did not improve from 0.82056
Epoch 81/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1307 - acc: 0.4353 - val_loss: 0.0210 - val_acc: 0.2158

Epoch 00081: val_acc did not improve from 0.82056
Epoch 82/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1344 - acc: 0.4356 - val_loss: 0.0247 - val_acc: 0.2139

Epoch 00082: val_acc did not improve from 0.82056
Epoch 83/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1165

138/138 [==============================] - 4s 28ms/step - loss: 0.0889 - acc: 0.4583 - val_loss: 0.0113 - val_acc: 0.2299

Epoch 00122: val_acc did not improve from 0.82056
Epoch 123/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0783 - acc: 0.4618 - val_loss: 0.0087 - val_acc: 0.2305

Epoch 00123: val_acc did not improve from 0.82056
Epoch 124/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0827 - acc: 0.4627 - val_loss: 0.0130 - val_acc: 0.2254

Epoch 00124: val_acc did not improve from 0.82056
Epoch 125/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0672 - acc: 0.4593 - val_loss: 0.0149 - val_acc: 0.2178

Epoch 00125: val_acc did not improve from 0.82056
Epoch 126/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0842 - acc: 0.4499 - val_loss: 0.0119 - val_acc: 0.2292

Epoch 00126: val_acc did not improve from 0.82056
Epoch 127/200
138/138 [==============================] - 4s 28ms/step - loss: 0

138/138 [==============================] - 4s 28ms/step - loss: 0.0615 - acc: 0.4631 - val_loss: 0.0132 - val_acc: 0.2171

Epoch 00166: val_acc did not improve from 0.82056
Epoch 167/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0699 - acc: 0.4532 - val_loss: 0.0072 - val_acc: 0.2261

Epoch 00167: val_acc did not improve from 0.82056
Epoch 168/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0599 - acc: 0.4616 - val_loss: 0.0077 - val_acc: 0.2146

Epoch 00168: val_acc did not improve from 0.82056
Epoch 169/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0588 - acc: 0.4584 - val_loss: 0.0127 - val_acc: 0.2241

Epoch 00169: val_acc did not improve from 0.82056
Epoch 170/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0669 - acc: 0.4593 - val_loss: 0.0059 - val_acc: 0.2171

Epoch 00170: val_acc did not improve from 0.82056
Epoch 171/200
138/138 [==============================] - 4s 28ms/step - loss: 0


Epoch 00001: val_acc improved from -inf to 0.00958, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/post_padding/1/weights-improvement-001-0.0096.hdf5
Epoch 2/200
138/138 [==============================] - 4s 28ms/step - loss: 0.8296 - acc: 0.1271 - val_loss: 0.3581 - val_acc: 0.0964

Epoch 00002: val_acc improved from 0.00958 to 0.09642, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/post_padding/1/weights-improvement-002-0.0964.hdf5
Epoch 3/200
138/138 [==============================] - 4s 27ms/step - loss: 0.8612 - acc: 0.1687 - val_loss: 0.3865 - val_acc: 0.1003

Epoch 00003: val_acc improved from 0.09642 to 0.10026, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/post_padding/1/weights-improvement-003-0.1003.hdf5
Epoch 4/200
138/138 [==============================] - 4s 27ms/step - loss: 0.8511 - acc: 0.1498 - val_loss: 0.

138/138 [==============================] - 4s 28ms/step - loss: 0.4451 - acc: 0.3107 - val_loss: 0.1091 - val_acc: 0.1928

Epoch 00031: val_acc did not improve from 0.19413
Epoch 32/200
138/138 [==============================] - 4s 28ms/step - loss: 0.4162 - acc: 0.3238 - val_loss: 0.1103 - val_acc: 0.2024

Epoch 00032: val_acc improved from 0.19413 to 0.20243, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/post_padding/1/weights-improvement-032-0.2024.hdf5
Epoch 33/200
138/138 [==============================] - 4s 28ms/step - loss: 0.3997 - acc: 0.3343 - val_loss: 0.1020 - val_acc: 0.2018

Epoch 00033: val_acc did not improve from 0.20243
Epoch 34/200
138/138 [==============================] - 4s 28ms/step - loss: 0.3767 - acc: 0.3352 - val_loss: 0.0881 - val_acc: 0.2063

Epoch 00034: val_acc improved from 0.20243 to 0.20626, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/post_p

138/138 [==============================] - 4s 28ms/step - loss: 0.1062 - acc: 0.4461 - val_loss: 0.0150 - val_acc: 0.2292

Epoch 00065: val_acc did not improve from 0.24393
Epoch 66/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1079 - acc: 0.4512 - val_loss: 0.0167 - val_acc: 0.2152

Epoch 00066: val_acc did not improve from 0.24393
Epoch 67/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1011 - acc: 0.4436 - val_loss: 0.0125 - val_acc: 0.2254

Epoch 00067: val_acc did not improve from 0.24393
Epoch 68/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0934 - acc: 0.4502 - val_loss: 0.0191 - val_acc: 0.2165

Epoch 00068: val_acc did not improve from 0.24393
Epoch 69/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0976 - acc: 0.4501 - val_loss: 0.0125 - val_acc: 0.2235

Epoch 00069: val_acc did not improve from 0.24393
Epoch 70/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0994

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



138/138 [==============================] - 4s 28ms/step - loss: 0.0817 - acc: 0.4542 - val_loss: 0.0133 - val_acc: 0.2420

Epoch 00084: val_acc did not improve from 0.24713
Epoch 85/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0852 - acc: 0.4604 - val_loss: 0.0144 - val_acc: 0.2356

Epoch 00085: val_acc did not improve from 0.24713
Epoch 86/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0797 - acc: 0.4622 - val_loss: 0.0200 - val_acc: 0.2331

Epoch 00086: val_acc did not improve from 0.24713
Epoch 87/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0813 - acc: 0.4627 - val_loss: 0.0115 - val_acc: 0.2305

Epoch 00087: val_acc did not improve from 0.24713
Epoch 88/200
 13/138 [=>............................] - ETA: 3s - loss: 0.0373 - acc: 0.2536